In [2]:
from dataset import SFMNNDataset
from network import SFMNNEncoder
from simulate import SFMNNSimulation
from loss import SFMNNLoss
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
import json
from dotenv import load_dotenv
import logging
import numpy as np

In [3]:
load_dotenv()
data_folder = os.getenv("DATA_FOLDER")

logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s [%(levelname)s] %(message)s",
                    handlers=[logging.StreamHandler()])
logger = logging.getLogger(__name__)

with open(data_folder + "simulation_sim_0_amb_0.json") as f:
    data = json.load(f)
with open(data_folder + "simulation_lookuptable.json") as f:
    lookup = json.load(f)

dataset = SFMNNDataset(
    lookup_table='output/simulation_lookuptable.json', 
    data_folder='output/', 
    patch_size=5,
)
dataloader = DataLoader(dataset, batch_size=3, shuffle=True, drop_last=False)


2025-03-22 01:48:18,111 [INFO] Using 3620 wavelength channels.
2025-03-22 01:48:18,114 [INFO] Found 1350 simulation JSON files.
Loading JSON files: 100%|██████████| 1350/1350 [00:16<00:00, 83.06it/s]
2025-03-22 01:48:34,745 [INFO] Fixed Esun computed as: 268.4315
2025-03-22 01:48:34,842 [INFO] Total simulations loaded: 1350
2025-03-22 01:48:34,843 [INFO] Total patches loaded: 60


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
# Number of spectral bands
n_spectral_bands = len(dataset.get_wl())
n_spectral_bands

3620

In [6]:
C,H,W = dataset[0].shape
H, W, C

(5, 5, 3623)

In [7]:
# Initialize the network
encoder = SFMNNEncoder(input_channels=C,num_variables = 8 ,latent_dim=4096).to(device)

print(encoder)

SFMNNEncoder(
  (input_norm): BatchNorm1d(3623, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc_layers): Sequential(
    (0): Linear(in_features=3623, out_features=4096, bias=True)
    (1): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=4096, out_features=8192, bias=True)
    (5): BatchNorm1d(8192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.1, inplace=False)
    (8): Linear(in_features=8192, out_features=8192, bias=True)
    (9): BatchNorm1d(8192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Linear(in_features=8192, out_features=8192, bias=True)
    (12): BatchNorm1d(8192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU()
    (14): Linear(in_features=8192, out_features=8192, bias=True)
    (15): BatchNorm1d(8192, ep

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

class FourStreamSimulator(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, t1, t2, t3, t6, t7, t8, t9, t10, t11, R, F):
        # Simplified toy model for LTOA.
        LTOA = t1 * t2 + (t1 * t8 * R + t9 * R + t10 * R + t11 * R + t6 * F + t7 * F) / (1 - t3 * R + 1e-6)
        return LTOA

# --- HyPlantSensorSimulator ---
class HyPlantSensorSimulator(nn.Module):
    def __init__(self, sensor_wavelengths):
        super().__init__()
        self.register_buffer('sensor_wavelengths', sensor_wavelengths)
        # Compute high_res as the spacing
        self.high_res = (sensor_wavelengths.max() - sensor_wavelengths.min()) / len(sensor_wavelengths)
        self.register_buffer('wl_range', torch.tensor([sensor_wavelengths.min(), sensor_wavelengths.max()]))
        # Define a high-resolution wavelength grid matching the sensor wavelengths length.
        self.register_buffer('lambda_hr', torch.linspace(self.wl_range[0].item(),
                                                           self.wl_range[1].item(),
                                                           len(sensor_wavelengths),
                                                           device=sensor_wavelengths.device))
        self.register_buffer('sensor_wavelengths', sensor_wavelengths.view(-1))



        # Fixed sensor miscalibration (in nm)
        self.fixed_delta_lambda = 0.04226162119141463

    def forward(self, L_hr, delta_lambda, delta_sigma):
        # L_hr is assumed to be of shape [B, H, W, spec] where spec ~241
        B, H, W, spec = L_hr.shape
        # Bring spectral axis to channel dimension: [B, spec, H, W]
        L_hr = L_hr.permute(0, 3, 1, 2)
        
        # --- 1. Convolve high-res spectrum with Gaussian SRF ---
        sigma = (0.27 + delta_sigma) * 2.3548  # Convert FWHM to sigma
        sigma_val = sigma.item() if torch.is_tensor(sigma) else sigma
        kernel_size = int(6 * sigma_val / self.high_res)
        if kernel_size % 2 == 0:
            kernel_size += 1
        x = torch.linspace(-3 * sigma_val, 3 * sigma_val, kernel_size, device=L_hr.device)
        kernel = torch.exp(-0.5 * (x / sigma_val)**2)
        kernel = kernel / kernel.sum()
        # Reshape for conv1d
        L_hr_reshaped = L_hr.permute(0, 2, 3, 1).reshape(B*H*W, 1, spec)
        L_blur = F.conv1d(L_hr_reshaped, kernel.view(1,1,kernel_size), padding=kernel_size//2)
        L_blur = L_blur.reshape(B, H, W, spec).permute(0, 3, 1, 2)
        
        # --- 2. Integrate (sample) the blurred spectrum at sensor wavelengths ---
        # Effective sensor wavelengths are shifted by fixed_delta_lambda.
        effective_sensor_wl = self.sensor_wavelengths + self.fixed_delta_lambda

        # Build Gaussian response matrix using lambda_hr:
        diff = self.lambda_hr.unsqueeze(0) - effective_sensor_wl.unsqueeze(1)  # [num_sensor, spec]
        sigma_prime = sigma_val  # Using the same sigma for response
        g = torch.exp(-0.5 * (diff / sigma_prime)**2)
        g = g / (g.sum(dim=1, keepdim=True) + 1e-6)  # [num_sensor, spec]
        
        # Integrate: result shape [B, num_sensor, H, W]
        L_hyp = torch.einsum('ij,bjhw->bihw', g, L_hr) * self.high_res
        return L_hyp

# --- SFMNNSimulation ---
class SFMNNSimulation(nn.Module):
    def __init__(self, sensor_wavelengths):
        super().__init__()
        self.four_stream = FourStreamSimulator()
        self.sensor_sim = HyPlantSensorSimulator(sensor_wavelengths)

    def forward(self, t1, t2, t3, t4, t5, t6, R, F, delta_lambda, delta_sigma):
        # Compute auxiliary variables as before.
        t7 = t3 * t4
        t8 = t3 * t6
        t9 = t4 * t5
        t10 = t4 * t2
        t11 = t3 * t2
        L_hr = self.four_stream(t1, t2, t3, t6, t7, t8, t9, t10, t11, R, F)
        L_hr = L_hr.permute(0, 2, 3, 1)
        L_hyp = self.sensor_sim(L_hr, delta_lambda, delta_sigma)
        # Optionally incorporate E_s here if needed.
        return L_hyp


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SFMNNLoss(nn.Module):
    def __init__(self, gamma_f=5, gamma_N=10, ndvi_threshold=0.15,
                 red_idx=680, nir_idx=800):
        """
        gamma_f: weight for the signal/florescence regularization
        gamma_N: weight for the NDVI-based physiological constraint
        ndvi_threshold: threshold for NDVI below which fluorescence is suppressed
        red_idx: index (or nearest index) corresponding to 680 nm (red)
        nir_idx: index (or nearest index) corresponding to 800 nm (NIR)
        """
        super().__init__()
        self.gamma_f = gamma_f
        self.gamma_N = gamma_N
        self.ndvi_threshold = ndvi_threshold
        self.red_idx = red_idx
        self.nir_idx = nir_idx

    def forward(self, pred, target, outputs, loss_indices):
        """
        Args:
          pred: [B, window_size, H, W], predicted LTOA over the spectral window
          target: [B, window_size, H, W], measured LTOA over the same window
          outputs: dict containing:
            'reflectance': [B, full_spectrum, H, W]
            'sif': [B, H, W], predicted fluorescence (can be negative without activation)
          loss_indices: spectral indices used for 'pred'/'target'
        Returns:
          total_loss, loss_dict
        """
        # 1) Reconstruction loss (L_w)
        L_w = torch.mean((pred - target) ** 2)

        # 2) NDVI-based physiological loss (L_ndvi)
        # Extract red and NIR channels from the full reflectance
        red = outputs['reflectance'][:, self.red_idx]  # [B, H, W]
        nir = outputs['reflectance'][:, self.nir_idx]    # [B, H, W]
        ndvi = (nir - red) / (nir + red + 1e-6)
        # Use ReLU on the predicted SIF so that only nonnegative fluorescence contributes
        L_ndvi = torch.mean(F.relu(outputs['sif']) * (ndvi <= self.ndvi_threshold).float())

        # 3) Signal/florescence regularization (L_f)
        w_f = self.compute_w_f(target, outputs['sif'])  # shape: [window_size]
        signal_loss = torch.mean((pred - target) ** 2 * w_f.view(1, -1, 1, 1))

        total_loss = L_w + self.gamma_f * signal_loss + self.gamma_N * L_ndvi

        loss_dict = {
            'total': total_loss,
            'recon': L_w,
            'signal': signal_loss,
            'ndvi': L_ndvi
        }
        return total_loss, loss_dict

    def compute_w_f(self, L_windowed, sif_pred, epsilon=1e-6):
        """
        Compute per-wavelength weights based on the predicted fluorescence.
        L_windowed: [B, window_size, H, W]
        sif_pred: [B, H, W]
        Returns:
          w_f: tensor of shape [window_size]
        """
        # Compute squared fluorescence, then average over batch and spatial dims.
        f_sq = sif_pred.unsqueeze(1) ** 2          # [B, 1, H, W]
        sum_f_sq = torch.mean(f_sq, dim=(0, 2, 3))   # scalar
        sigma_L2 = torch.var(L_windowed, dim=(0, 2, 3)) + 1e-3  # [window_size]
        numerator = sum_f_sq  # scalar
        denominator = sum_f_sq / (sigma_L2 + epsilon)  # [window_size]
        w_f = (1.0 / sigma_L2) * (sum_f_sq / (denominator.sum() + 1e-6))
        w_f = w_f / (w_f.sum() + 1e-6)  # Normalize weights to sum to 1
        return w_f


In [ ]:
def train_model(encoder):
    from dotenv import load_dotenv
    import os
    import torch
    import torch.optim as optim
    from tqdm import tqdm  # Import tqdm for progress bars

    load_dotenv()
    data_folder = os.getenv("DATA_FOLDER")
    lookup_table_path = os.path.join(data_folder, 'simulation_lookuptable.json')
    # dataset = SFMNNDataset(lookup_table_path, data_folder, patch_size=5)
    # For demonstration, assume dataset is already defined elsewhere.

    dataloader = torch.utils.data.DataLoader(dataset, batch_size=3, shuffle=True, drop_last=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Using device: {device}")

    n_spectral_bands = len(dataset.get_wl())
    C = n_spectral_bands + 3
    logger.info(f"Spectral channels: {n_spectral_bands}, total input channels: {C}")

    optimizer = optim.Adam(encoder.parameters(), lr=1e-5, weight_decay=1e-5)

    # Initialize the loss function (atmospheric term removed)
    loss_fn = SFMNNLoss(red_idx=680, nir_idx=850, gamma_f=5, gamma_N=10, ndvi_threshold=0.15).to(device)

    sensor_wl = torch.tensor(dataset.get_wl(), dtype=torch.float, device=device)
    simulation = SFMNNSimulation(sensor_wl).to(device)

    fixed_esun = dataset.fixed_esun
    logger.info(f"Using fixed Esun: {fixed_esun:.4f}")

    # Define spectral window indices for loss (e.g., wavelengths between 760 and 780 nm)
    indices = ((sensor_wl >= 760.0) & (sensor_wl <= 780.0)).nonzero(as_tuple=True)[0]
    logger.info(f"Loss calculated over indices: {indices.tolist()}, corresponding to {sensor_wl[indices[0]].item():.1f}-{sensor_wl[indices[-1]].item():.1f} nm.")

    num_epochs = 1000
    for epoch in range(num_epochs):
        encoder.train()
        epoch_loss = 0.0
        num_samples = 0

        # Wrap the dataloader with tqdm for a progress bar at the batch level
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
        for batch in pbar:
            batch = batch.to(device)
            B, _, H, W = batch.shape
            n_spectral_bands = len(dataset.get_wl())

            # Extract target reflectance: first n_spectral_bands channels.
            target_full = batch[:, :n_spectral_bands, :, :]
            target = target_full[:, indices, :, :]  

            latent = encoder(batch)  # Expected shape: [B, H, W, 9, latent_dim]
            latent = latent[:, :, :, :, :n_spectral_bands]

            t1 = latent[:, :, :, 0, :].permute(0, 3, 1, 2)
            t2 = latent[:, :, :, 1, :].permute(0, 3, 1, 2)
            t3 = latent[:, :, :, 2, :].permute(0, 3, 1, 2)
            t4 = latent[:, :, :, 3, :].permute(0, 3, 1, 2)
            t5 = latent[:, :, :, 4, :].permute(0, 3, 1, 2)
            t6 = latent[:, :, :, 5, :].permute(0, 3, 1, 2)
            R_param = latent[:, :, :, 6, :].permute(0, 3, 1, 2)
            F_param = latent[:, :, :, 7, :].permute(0, 3, 1, 2)

            # For delta_lambda and delta_sigma, pass 0.0 (delta_lambda is ignored in simulation)
            sim_output_full = simulation(
                t1, t2, t3, t4, t5, t6, R_param, F_param,
                torch.tensor(0.0, device=device),  # delta_lambda
                torch.tensor(0.0, device=device),  # delta_sigma
            )
            # sim_output_full: [B, n_spectral_bands, H, W]
            pred = sim_output_full[:, indices, :, :]

            # Predicted SIF from F_param.
            predicted_sif = F_param.mean(dim=1)  # [B, H, W]
            outputs_dict = {
                'reflectance': sim_output_full,
                'sif': predicted_sif
            }

            # Calculate the loss using the loss function with (pred, target, outputs, indices)
            total_loss, loss_dict = loss_fn(pred, target, outputs_dict, indices)
            optimizer.zero_grad()
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(encoder.parameters(), max_norm=1.0)
            optimizer.step()

            epoch_loss += total_loss.item() * B
            num_samples += B

            # Update the progress bar with loss values.
            pbar.set_postfix({
                "loss": total_loss.item(),
                "recon": loss_dict['recon'].item(),
                "signal": loss_dict['signal'].item(),
                "ndvi": loss_dict['ndvi'].item()
            })
        avg_epoch_loss = epoch_loss / num_samples
        tqdm.write(f"Epoch {epoch+1}/{num_epochs} - Avg Loss: {avg_epoch_loss:.4f}")

    return encoder

# Example usage:
encoder = train_model(encoder)


2025-03-22 01:48:38,225 [INFO] Using device: cuda
2025-03-22 01:48:38,226 [INFO] Spectral channels: 3620, total input channels: 3623
2025-03-22 01:48:38,760 [INFO] Using fixed Esun: 268.4315
2025-03-22 01:48:38,773 [INFO] Loss calculated over indices: [2227, 2228, 2229, 2230, 2231, 2232, 2233, 2234, 2235, 2236, 2237, 2238, 2239, 2240, 2241, 2242, 2243, 2244, 2245, 2246, 2247, 2248, 2249, 2250, 2251, 2252, 2253, 2254, 2255, 2256, 2257, 2258, 2259, 2260, 2261, 2262, 2263, 2264, 2265, 2266, 2267, 2268, 2269, 2270, 2271, 2272, 2273, 2274, 2275, 2276, 2277, 2278, 2279, 2280, 2281, 2282, 2283, 2284, 2285, 2286, 2287, 2288, 2289, 2290, 2291, 2292, 2293, 2294, 2295, 2296, 2297, 2298, 2299, 2300, 2301, 2302, 2303, 2304, 2305, 2306, 2307, 2308, 2309, 2310, 2311, 2312, 2313, 2314, 2315, 2316, 2317, 2318, 2319, 2320, 2321, 2322, 2323, 2324, 2325, 2326, 2327, 2328, 2329, 2330, 2331, 2332, 2333, 2334, 2335, 2336, 2337, 2338, 2339, 2340, 2341, 2342, 2343, 2344, 2345, 2346, 2347, 2348, 2349, 2350, 235

Epoch 1/1000 - Avg Loss: 10595.2425


Epoch 2/1000 - Avg Loss: 10588.7444


Epoch 3/1000 - Avg Loss: 10495.4362


Epoch 4/1000 - Avg Loss: 10685.6029


...

Epoch 996/1000 - Avg Loss: 892.6584


Epoch 997/1000 - Avg Loss: 902.0457


Epoch 998/1000 - Avg Loss: 971.8833


Epoch 999/1000 - Avg Loss: 1004.9675


Epoch 1000/1000 - Avg Loss: 902.7690


In [ ]:
import pickle
with open('output/encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)

In [20]:
import random
import torch
import numpy as np
import plotly.graph_objects as go

def verify_inference(encoder, simulation, dataset, sensor_wl, device):
    """
    Randomly selects a patch from the dataset, performs inference,
    and creates interactive Plotly plots for:
      - The target reflectance spectrum (from the patch)
      - The raw predicted LTOA spectrum (before sensor simulation)
      - The simulated sensor output L_hyp spectrum (after sensor simulation)
      - The full predicted fluorescence (SIF) spectral signature

    Assumes:
      - Dataset returns tensor of shape [C, H, W] (or (tensor, index) if return_index=True)
        with C = num_wavelengths + 3.
      - sensor_wl is a tensor of shape [num_wavelengths].
      - Extra channels are ordered: XTE, SZA, GNDALT.
    """
    encoder.eval()
    simulation.eval()

    # Randomly select one patch.
    idx = random.randint(0, len(dataset) - 1)
    sample = dataset[idx]
    # If dataset returns a tuple, take the first element.
    if isinstance(sample, tuple):
        sample = sample[0]
    sample = sample.unsqueeze(0).to(device)  # shape: [1, C, H, W]

    B, C, H, W = sample.shape
    n_spectral = len(dataset.get_wl())

    # Extract target reflectance from the first n_spectral channels.
    target_full = sample[:, :n_spectral, :, :]

    with torch.no_grad():
        latent = encoder(sample)  # Expected shape: [B, H, W, 9, latent_dim]

        # Extract latent variables; each becomes shape: [B, n_spectral, H, W]
        t1 = latent[:, :, :, 0, :].permute(0, 3, 1, 2)
        t2 = latent[:, :, :, 1, :].permute(0, 3, 1, 2)
        t3 = latent[:, :, :, 2, :].permute(0, 3, 1, 2)
        t4 = latent[:, :, :, 3, :].permute(0, 3, 1, 2)
        t5 = latent[:, :, :, 4, :].permute(0, 3, 1, 2)
        t6 = latent[:, :, :, 5, :].permute(0, 3, 1, 2)
        R_param = latent[:, :, :, 6, :].permute(0, 3, 1, 2)
        F_param = latent[:, :, :, 7, :].permute(0, 3, 1, 2)
        t7 = t3 * t4
        t8 = t3 * t6
        t9 = t4 * t5
        t10 = t4 * t2
        t11 = t3 * t2
        
        # Compute simulated sensor output L_hyp.
        # CRITICAL: Pass n_spectral here, NOT len(sensor_wl).
        L_hyp = simulation(t1, t2, t3, t4, t5, t6, R_param, F_param,
                           torch.tensor(0.042, device=device),  # delta_lambda
                           torch.tensor(0.3, device=device))    # delta_sigma

        # Also compute raw predicted LTOA by calling four_stream directly.
        L_toa = simulation.four_stream(t1, t2, t3, t6, t7, t8, t9, t10, t11, R_param, F_param)
        predicted_sif = F_param  # [B, n_spectral, H, W]

    # Select the center pixel for comparison.
    center_h, center_w = H // 2, W // 2
    target_spec = target_full[0, :, center_h, center_w].cpu().numpy()
    L_toa_spec = L_toa[0, :, center_h, center_w].cpu().numpy()
    L_hyp_spec = L_hyp[0, :, center_h, center_w].cpu().numpy()
    predicted_sif_spec = predicted_sif[0, :, center_h, center_w].cpu().numpy()
    wavelengths = sensor_wl.cpu().numpy()

    # Plot 1: Spectrum Comparison (Target, LTOA, L_hyp)
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x=wavelengths, y=target_spec, mode='lines',
                              name='Target Reflectance', line=dict(width=2)))
    fig1.add_trace(go.Scatter(x=wavelengths, y=L_toa_spec, mode='lines',
                              name='Predicted LTOA (raw)', line=dict(width=2, dash='dash')))
    fig1.add_trace(go.Scatter(x=wavelengths, y=L_hyp_spec, mode='lines',
                              name='Simulated Sensor Output (L_hyp)', line=dict(width=2, dash='dot')))
    fig1.update_layout(title='Center Pixel Spectrum Comparison',
                       xaxis_title='Wavelength (nm)',
                       yaxis_title='Radiance / Reflectance',
                       template="plotly_white")
    fig1.show()

    # Plot 2: Predicted Fluorescence (SIF)
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=wavelengths, y=predicted_sif_spec, mode='lines',
                              name='Predicted Fluorescence (SIF)', line=dict(width=2)))
    fig2.update_layout(title='Predicted Fluorescence Spectrum at Center Pixel',
                       xaxis_title='Wavelength (nm)',
                       yaxis_title='Fluorescence Intensity',
                       template="plotly_white")
    fig2.show()

    # Plot 3: Predicted LTOA Spectrum (raw)
    fig3 = go.Figure()
    fig3.add_trace(go.Scatter(x=wavelengths, y=L_toa_spec, mode='lines',
                              name='Predicted LTOA (raw)', line=dict(width=2, dash='dash')))
    fig3.update_layout(title='Predicted LTOA Spectrum at Center Pixel',
                       xaxis_title='Wavelength (nm)',
                       yaxis_title='LTOA',
                       template="plotly_white")
    fig3.show()

    # Plot 4: Simulated Sensor Output (L_hyp)
    fig4 = go.Figure()
    fig4.add_trace(go.Scatter(x=wavelengths, y=L_hyp_spec, mode='lines',
                              name='Simulated Sensor Output (L_hyp)', line=dict(width=2, dash='dot')))
    fig4.update_layout(title='Predicted L_hyp Spectrum at Center Pixel',
                       xaxis_title='Wavelength (nm)',
                       yaxis_title='L_hyp',
                       template="plotly_white")
    fig4.show()

    # Print out center pixel values
    print("Center Pixel Target Reflectance:", target_spec)
    print("Center Pixel Raw LTOA:", L_toa_spec)
    print("Center Pixel Simulated L_hyp:", L_hyp_spec)
    print("Center Pixel Predicted SIF Spectrum:", predicted_sif_spec)


# Example usage:
# Assume `encoder`, `simulation`, `dataset`, and `device` are defined elsewhere.
# sensor_wl is a tensor containing wavelengths from the dataset.
sensor_wl = torch.tensor(dataset.get_wl(), dtype=torch.float, device=device)
simulation = SFMNNSimulation(sensor_wl).to(device)

verify_inference(encoder, simulation, dataset, sensor_wl, device)


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

----
# <center>Playground, this is for testing purposes